<a href="https://colab.research.google.com/github/MuhammadIrzam447/NewEncodings/blob/main/Train_13.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Notebook to Train and Evaluate ResNet and ViT on 100 and __% datasets residing in a single directory.

# Load Data


In [ ]:
# import os

# # Define the path to your dataset folder
# dataset_folder = '/content/MMLearning/data/food-101/flip/multimodal_img_et_flip/test'

# # Define the path for the output text file
# output_file = '/content/MMLearning/data/food-101/flip/multimodal_img_et_flip/test_dataset.txt'

# # Open the output file for writing
# with open(output_file, 'w') as f:
#     # Iterate through the subdirectories in the dataset folder
#     for label in os.listdir(dataset_folder):
#         label_path = os.path.join(dataset_folder, label)

#         # Check if it's a directory (class folder)
#         if os.path.isdir(label_path):
#             # Get the label (class name) from the folder name
#             class_name = label

#             # Iterate through the images in the class folder
#             for image_file in os.listdir(label_path):
#                 image_path = os.path.join(label_path, image_file)

#                 # Write the image path and label to the output file
#                 f.write(f'{image_path}, {class_name}\n')


Train Dataset

In [1]:
import os

image_file_paths = []
image_genre_labels = []
enc_file_paths = []
enc_genre_labels = []

labels_file = "/content/MMLearning/data/food-101/flip/multimodal_img_et_flip/train_dataset.txt"

with open(labels_file, 'r') as file:
    for line in file:
        parts = line.strip().split(', ')
        filename = parts[0].strip()
        labels = parts[1].strip()

        if filename.endswith("_3.jpg"):
            enc_file_paths.append(filename)
            enc_genre_labels.append(labels)

        elif filename.endswith("_4.jpg"):
            image_file_paths.append(filename)
            image_genre_labels.append(labels)

In [2]:
len(image_file_paths), len(enc_file_paths)

(68051, 68051)

In [3]:
image_file_paths[0:5] , image_genre_labels[0:5]

(['/content/MMLearning/data/food-101/flip/multimodal_img_et_flip/train/french_onion_soup/french_onion_soup_335_4.jpg',
  '/content/MMLearning/data/food-101/flip/multimodal_img_et_flip/train/french_onion_soup/french_onion_soup_834_4.jpg',
  '/content/MMLearning/data/food-101/flip/multimodal_img_et_flip/train/french_onion_soup/french_onion_soup_489_4.jpg',
  '/content/MMLearning/data/food-101/flip/multimodal_img_et_flip/train/french_onion_soup/french_onion_soup_98_4.jpg',
  '/content/MMLearning/data/food-101/flip/multimodal_img_et_flip/train/french_onion_soup/french_onion_soup_592_4.jpg'],
 ['french_onion_soup',
  'french_onion_soup',
  'french_onion_soup',
  'french_onion_soup',
  'french_onion_soup'])

In [4]:
enc_file_paths[0:5] , enc_genre_labels[0:5]

(['/content/MMLearning/data/food-101/flip/multimodal_img_et_flip/train/french_onion_soup/french_onion_soup_124_3.jpg',
  '/content/MMLearning/data/food-101/flip/multimodal_img_et_flip/train/french_onion_soup/french_onion_soup_37_3.jpg',
  '/content/MMLearning/data/food-101/flip/multimodal_img_et_flip/train/french_onion_soup/french_onion_soup_690_3.jpg',
  '/content/MMLearning/data/food-101/flip/multimodal_img_et_flip/train/french_onion_soup/french_onion_soup_240_3.jpg',
  '/content/MMLearning/data/food-101/flip/multimodal_img_et_flip/train/french_onion_soup/french_onion_soup_835_3.jpg'],
 ['french_onion_soup',
  'french_onion_soup',
  'french_onion_soup',
  'french_onion_soup',
  'french_onion_soup'])

In [5]:
unique_labels = set(enc_genre_labels)
unique_labels_list = list(unique_labels)

print("Unique Labels:", unique_labels_list)

Unique Labels: ['guacamole', 'chicken_quesadilla', 'french_fries', 'beef_tartare', 'chocolate_cake', 'panna_cotta', 'paella', 'chocolate_mousse', 'peking_duck', 'cheesecake', 'greek_salad', 'breakfast_burrito', 'red_velvet_cake', 'beet_salad', 'lobster_roll_sandwich', 'frozen_yogurt', 'pad_thai', 'oysters', 'escargots', 'prime_rib', 'clam_chowder', 'bread_pudding', 'fried_calamari', 'ravioli', 'gnocchi', 'beignets', 'pulled_pork_sandwich', 'baby_back_ribs', 'pizza', 'bruschetta', 'dumplings', 'chicken_curry', 'nachos', 'creme_brulee', 'lasagna', 'hot_and_sour_soup', 'huevos_rancheros', 'pork_chop', 'samosa', 'caprese_salad', 'grilled_salmon', 'omelette', 'spaghetti_carbonara', 'edamame', 'takoyaki', 'baklava', 'poutine', 'donuts', 'ceviche', 'miso_soup', 'spaghetti_bolognese', 'cannoli', 'sushi', 'sashimi', 'garlic_bread', 'falafel', 'risotto', 'club_sandwich', 'cheese_plate', 'bibimbap', 'fish_and_chips', 'spring_rolls', 'macarons', 'strawberry_shortcake', 'pancakes', 'fried_rice', 't

In [6]:
num_classes = len(unique_labels_list)
num_classes

101

In [7]:
import random


percentage = 0.3

label_counts = {label: 0 for label in unique_labels_list}

num_samples_to_include_per_label = {}
for label in unique_labels_list:
    label_count = enc_genre_labels.count(label)
    num_samples_to_include = int(label_count * percentage)
    num_samples_to_include_per_label[label] = num_samples_to_include


filtered_enc_file_paths = []
filtered_enc_genre_labels = []


selected_samples_per_label = {label: 0 for label in unique_labels_list}
for path, label in zip(enc_file_paths, enc_genre_labels):
    if selected_samples_per_label[label] < num_samples_to_include_per_label[label]:
        filtered_enc_file_paths.append(path)
        filtered_enc_genre_labels.append(label)
        selected_samples_per_label[label] += 1

In [8]:
len(filtered_enc_file_paths)

20365

In [9]:
# Create new lists to combine data from both lists
train_file_paths = image_file_paths + filtered_enc_file_paths
train_genre_labels = image_genre_labels + filtered_enc_genre_labels

In [10]:
len(train_file_paths)

88416

Test Dataset

In [11]:
import os

test_image_file_paths = []
test_image_genre_labels = []
test_enc_file_paths = []
test_enc_genre_labels = []

test_labels_file = "/content/MMLearning/data/food-101/flip/multimodal_img_et_flip/test_dataset.txt"

with open(test_labels_file, 'r') as file:
    for line in file:
        parts = line.strip().split(', ')
        filename = parts[0].strip()
        labels = parts[1].strip()

        if filename.endswith("_3.jpg"):
            test_enc_file_paths.append(filename)
            test_enc_genre_labels.append(labels)

        elif filename.endswith("_4.jpg"):
            test_image_file_paths.append(filename)
            test_image_genre_labels.append(labels)

In [12]:
len(test_image_file_paths), len(test_enc_file_paths)

(22716, 22716)

In [13]:
test_image_file_paths[0:5] , test_image_genre_labels[0:5]

(['/content/MMLearning/data/food-101/flip/multimodal_img_et_flip/test/french_onion_soup/french_onion_soup_400_4.jpg',
  '/content/MMLearning/data/food-101/flip/multimodal_img_et_flip/test/french_onion_soup/french_onion_soup_719_4.jpg',
  '/content/MMLearning/data/food-101/flip/multimodal_img_et_flip/test/french_onion_soup/french_onion_soup_163_4.jpg',
  '/content/MMLearning/data/food-101/flip/multimodal_img_et_flip/test/french_onion_soup/french_onion_soup_423_4.jpg',
  '/content/MMLearning/data/food-101/flip/multimodal_img_et_flip/test/french_onion_soup/french_onion_soup_243_4.jpg'],
 ['french_onion_soup',
  'french_onion_soup',
  'french_onion_soup',
  'french_onion_soup',
  'french_onion_soup'])

In [14]:
import random

label_counts = {label: 0 for label in unique_labels_list}

test_num_samples_to_include_per_label = {}
for label in unique_labels_list:
    label_count = test_enc_genre_labels.count(label)
    num_samples_to_include = int(label_count * percentage)
    test_num_samples_to_include_per_label[label] = num_samples_to_include


test_filtered_enc_file_paths = []
test_filtered_enc_genre_labels = []


test_selected_samples_per_label = {label: 0 for label in unique_labels_list}
for path, label in zip(test_enc_file_paths, test_enc_genre_labels):
    if test_selected_samples_per_label[label] < test_num_samples_to_include_per_label[label]:
        test_filtered_enc_file_paths.append(path)
        test_filtered_enc_genre_labels.append(label)
        test_selected_samples_per_label[label] += 1

In [15]:
len(test_filtered_enc_file_paths)

6770

In [16]:
# Create new lists to combine data from both lists
test_file_paths = test_image_file_paths + test_filtered_enc_file_paths
test_genre_labels = test_image_genre_labels + test_filtered_enc_genre_labels

In [17]:
len(test_file_paths)

29486

# ResNet

In [ ]:
from torch.utils.data import Dataset
from torchvision import transforms
from PIL import Image

class CustomDataset(Dataset):
    def __init__(self, file_paths, labels, transform=None):
        self.file_paths = file_paths
        self.labels = labels
        self.transform = transform

    def __len__(self):
        return len(self.file_paths)

    def __getitem__(self, idx):
        img_path = self.file_paths[idx]
        image = Image.open(img_path)
        label = int(self.labels[idx])
        # label = torch.tensor(label,dtype=torch.float32)

        if self.transform:
            image = self.transform(image)

        return image, label


transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

In [ ]:
train_dataset = CustomDataset(train_file_paths, train_genre_labels, transform=transform)
test_dataset = CustomDataset(test_file_paths, test_genre_labels, transform=transform)

In [ ]:
from torch.utils.data import DataLoader

batch_size = 32  # Adjust as needed
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import models
device = "cuda" if torch.cuda.is_available() else "cpu"

resnet = models.resnet101(pretrained=True)
num_features = resnet.fc.in_features
resnet.fc = nn.Linear(num_features, num_classes)  # num_classes is the number of classes in your dataset
resnet.to(device)
print(resnet)


In [ ]:
criterion = nn.CrossEntropyLoss()
# optimizer = optim.SGD(resnet.parameters(), lr=0.001, momentum=0.9)
optimizer = optim.Adam(resnet.parameters(), lr=0.001)
num_epochs = 30

In [ ]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, classification_report, roc_auc_score
import numpy as np

In [ ]:
training_loss = []

for epoch in range(num_epochs):
    resnet.train()
    running_loss = 0.0


    for images, labels in train_loader:
        images = images.to(device)
        labels = labels.to(device)

        optimizer.zero_grad()
        outputs = resnet(images)
        loss = criterion(outputs, labels)

        loss.backward()
        optimizer.step()

        running_loss += loss.item() * images.size(0)

    epoch_loss = running_loss / len(train_loader.dataset)
    training_loss.append(epoch_loss)

    print("Training Loss==========================>>")
    print(f"Epoch {epoch+1}/{num_epochs} Training Loss: {epoch_loss:.4f}")

    resnet.eval()
    predicted_classes = []
    actual_labels = []
    with torch.no_grad():
        for images, labels in test_loader:
            images = images.to(device)
            labels = labels.to(device)

            outputs = resnet(images)

            _, predicted_label = torch.max(outputs, 1)

            predicted_classes.extend(predicted_label.cpu().tolist())
            actual_labels.extend(labels.cpu().tolist())

    accuracy = accuracy_score(actual_labels, predicted_classes)
    # precision = precision_score(actual_labels, predicted_classes, average='weighted')
    # recall = recall_score(actual_labels, predicted_classes, average='weighted')
    # f1 = f1_score(actual_labels, predicted_classes, average='weighted')

    print("Accuracy:", accuracy)
    # print("Precision:", precision)
    # print("Recall:", recall)
    # print("F1-score:", f1)

    print(classification_report(actual_labels, predicted_classes))
    # print("Confusion Matrix: ")
    # cm = confusion_matrix(actual_labels, predicted_classes)
    # print(cm)
    predicted_classes = np.array(predicted_classes)
    actual_labels = np.array(actual_labels)
    print(roc_auc_score(actual_labels, predicted_classes))


    save_dir = "/content/Model/Models-Train-30/"
    os.makedirs(save_dir, exist_ok=True)  # Create the directory if it doesn't exist

    model_name = str(epoch+1) + "_model.pth"
    save_path = os.path.join(save_dir, model_name)  # Specify the complete path to the model file
    torch.save(resnet.state_dict(), save_path)

# ViT

In [ ]:
import torch
!pip install transformers evaluate datasets
import requests
import torch
from PIL import Image
from transformers import *
from tqdm import tqdm

In [ ]:
device = "cuda" if torch.cuda.is_available() else "cpu"

model_name = "google/vit-base-patch16-224"
image_processor = ViTImageProcessor.from_pretrained(model_name)
model = ViTForImageClassification.from_pretrained(model_name).to(device)

Label Encoding

In [20]:
from sklearn.preprocessing import LabelEncoder

label_encoder = LabelEncoder()
transformed_train_genre_labels = label_encoder.fit_transform(train_genre_labels)

In [ ]:
# label_to_id_mapping = {label: id for label, id in zip(label_encoder.classes_, label_encoder.transform(label_encoder.classes_))}
# label_to_id_mapping

In [21]:
label_encoder1 = LabelEncoder()
transformed_test_genre_labels = label_encoder1.fit_transform(test_genre_labels)

In [22]:
label_to_id_mapping = {label: id for label, id in zip(label_encoder1.classes_, label_encoder1.transform(label_encoder1.classes_))}
id_to_label_mapping = {id: label for id, label in enumerate(label_encoder1.classes_)}

In [23]:
# Convert int64 to int in id_to_label_mapping and label_to_id_mapping
id_to_label_mapping = {int(id): label for id, label in id_to_label_mapping.items()}
label_to_id_mapping = {label: int(id) for label, id in label_to_id_mapping.items()}

In [24]:
id_to_label_mapping

{0: 'apple_pie',
 1: 'baby_back_ribs',
 2: 'baklava',
 3: 'beef_carpaccio',
 4: 'beef_tartare',
 5: 'beet_salad',
 6: 'beignets',
 7: 'bibimbap',
 8: 'bread_pudding',
 9: 'breakfast_burrito',
 10: 'bruschetta',
 11: 'caesar_salad',
 12: 'cannoli',
 13: 'caprese_salad',
 14: 'carrot_cake',
 15: 'ceviche',
 16: 'cheese_plate',
 17: 'cheesecake',
 18: 'chicken_curry',
 19: 'chicken_quesadilla',
 20: 'chicken_wings',
 21: 'chocolate_cake',
 22: 'chocolate_mousse',
 23: 'churros',
 24: 'clam_chowder',
 25: 'club_sandwich',
 26: 'crab_cakes',
 27: 'creme_brulee',
 28: 'croque_madame',
 29: 'cup_cakes',
 30: 'deviled_eggs',
 31: 'donuts',
 32: 'dumplings',
 33: 'edamame',
 34: 'eggs_benedict',
 35: 'escargots',
 36: 'falafel',
 37: 'filet_mignon',
 38: 'fish_and_chips',
 39: 'foie_gras',
 40: 'french_fries',
 41: 'french_onion_soup',
 42: 'french_toast',
 43: 'fried_calamari',
 44: 'fried_rice',
 45: 'frozen_yogurt',
 46: 'garlic_bread',
 47: 'gnocchi',
 48: 'greek_salad',
 49: 'grilled_chees

In [25]:
len(train_genre_labels), len(transformed_train_genre_labels)

(88416, 88416)

In [26]:
len(test_genre_labels), len(transformed_test_genre_labels)

(29486, 29486)

Dataset Creating

In [27]:
from datasets import Dataset

train_data = {'image': train_file_paths, 'label': transformed_train_genre_labels}
ds_train = Dataset.from_dict(train_data)

In [28]:
val_data = {'image': test_file_paths, 'label': transformed_test_genre_labels}
ds_val = Dataset.from_dict(val_data)

In [29]:
labels = unique_labels_list
labels

['guacamole',
 'chicken_quesadilla',
 'french_fries',
 'beef_tartare',
 'chocolate_cake',
 'panna_cotta',
 'paella',
 'chocolate_mousse',
 'peking_duck',
 'cheesecake',
 'greek_salad',
 'breakfast_burrito',
 'red_velvet_cake',
 'beet_salad',
 'lobster_roll_sandwich',
 'frozen_yogurt',
 'pad_thai',
 'oysters',
 'escargots',
 'prime_rib',
 'clam_chowder',
 'bread_pudding',
 'fried_calamari',
 'ravioli',
 'gnocchi',
 'beignets',
 'pulled_pork_sandwich',
 'baby_back_ribs',
 'pizza',
 'bruschetta',
 'dumplings',
 'chicken_curry',
 'nachos',
 'creme_brulee',
 'lasagna',
 'hot_and_sour_soup',
 'huevos_rancheros',
 'pork_chop',
 'samosa',
 'caprese_salad',
 'grilled_salmon',
 'omelette',
 'spaghetti_carbonara',
 'edamame',
 'takoyaki',
 'baklava',
 'poutine',
 'donuts',
 'ceviche',
 'miso_soup',
 'spaghetti_bolognese',
 'cannoli',
 'sushi',
 'sashimi',
 'garlic_bread',
 'falafel',
 'risotto',
 'club_sandwich',
 'cheese_plate',
 'bibimbap',
 'fish_and_chips',
 'spring_rolls',
 'macarons',
 'str

In [30]:
import PIL.Image as pil

def transform(examples):
  inputs = image_processor([pil.open(img).convert("RGB") for img in examples["image"]], return_tensors="pt")
  inputs["labels"] = examples["label"]
  return inputs

In [31]:
train_dataset = ds_train.with_transform(transform)
val_dataset = ds_val.with_transform(transform)

In [32]:
train_dataset

Dataset({
    features: ['image', 'label'],
    num_rows: 88416
})

In [33]:
val_dataset

Dataset({
    features: ['image', 'label'],
    num_rows: 29486
})

In [34]:
for item in val_dataset:
  print(item['pixel_values'].shape)
  print(item["labels"])
  break

torch.Size([3, 224, 224])
41


In [35]:
import torch

def collate_fn(batch):
  return {
      "pixel_values": torch.stack([x["pixel_values"] for x in batch]),
      "labels": torch.tensor([x["labels"] for x in batch]),
  }

In [36]:
# load the ViT model
model = ViTForImageClassification.from_pretrained(
    model_name,
    num_labels=len(unique_labels_list),
    # id2label={str(i): c for i, c in enumerate(unique_labels_list)},
    id2label=id_to_label_mapping,
    label2id=label_to_id_mapping,
    # label2id={c: str(i) for i, c in enumerate(unique_labels_list)},
    ignore_mismatched_sizes=True,
)

loading configuration file config.json from cache at /root/.cache/huggingface/hub/models--google--vit-base-patch16-224/snapshots/3f49326eb077187dfe1c2a2bb15fbd74e6ab91e3/config.json
Model config ViTConfig {
  "_name_or_path": "google/vit-base-patch16-224",
  "architectures": [
    "ViTForImageClassification"
  ],
  "attention_probs_dropout_prob": 0.0,
  "encoder_stride": 16,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.0,
  "hidden_size": 768,
  "id2label": {
    "0": "apple_pie",
    "1": "baby_back_ribs",
    "2": "baklava",
    "3": "beef_carpaccio",
    "4": "beef_tartare",
    "5": "beet_salad",
    "6": "beignets",
    "7": "bibimbap",
    "8": "bread_pudding",
    "9": "breakfast_burrito",
    "10": "bruschetta",
    "11": "caesar_salad",
    "12": "cannoli",
    "13": "caprese_salad",
    "14": "carrot_cake",
    "15": "ceviche",
    "16": "cheese_plate",
    "17": "cheesecake",
    "18": "chicken_curry",
    "19": "chicken_quesadilla",
    "20": "chicken_wings",
    "21"

Evaluation Metric

In [37]:
from evaluate import load
import numpy as np
from sklearn.metrics import roc_auc_score

# load the accuracy and f1 metrics from the evaluate module
accuracy = load("accuracy")
f1 = load("f1")

def compute_metrics(eval_pred):
  # compute the accuracy and f1 scores & return them
  accuracy_score = accuracy.compute(predictions=np.argmax(eval_pred.predictions, axis=1), references=eval_pred.label_ids)
  f1_score = f1.compute(predictions=np.argmax(eval_pred.predictions, axis=1), references=eval_pred.label_ids, average="macro")

  # auroc_score = roc_auc_score(eval_pred.label_ids, np.argmax(eval_pred.predictions, axis=1))
  # print(f"AUROC Score: {auroc_score:.4f}")

  return {**accuracy_score, **f1_score}

Training

In [ ]:
# !pip install accelerate -U
# !pip install transformers[torch]

In [38]:
from transformers import TrainingArguments

training_args = TrainingArguments(
  output_dir="/content/MMLearning/data/Models/Model-13", # output directory
  per_device_train_batch_size=32, # batch size per device during training
  evaluation_strategy="steps",    # evaluation strategy to adopt during training
  num_train_epochs=40,             # total number of training epochs
  # fp16=True,                    # use mixed precision
  save_steps=9000,                # number of update steps before saving checkpoint
  eval_steps=9000,                # number of update steps before evaluating
  logging_steps=9000,             # number of update steps before logging
  # save_steps=50,
  # eval_steps=50,
  # logging_steps=50,
  save_total_limit=6,             # limit the total amount of checkpoints on disk
  remove_unused_columns=False,    # remove unused columns from the dataset
  push_to_hub=False,              # do not push the model to the hub
  report_to='tensorboard',        # report metrics to tensorboard
  load_best_model_at_end=True,    # load the best model at the end of training
)

Found safetensors installation, but --save_safetensors=False. Safetensors should be a preferred weights saving format due to security and performance reasons. If your model cannot be saved by safetensors please feel free to open an issue at https://github.com/huggingface/safetensors!
PyTorch: setting up devices


In [39]:
from transformers import Trainer

trainer = Trainer(
    model=model,                        # the instantiated 🤗 Transformers model to be trained
    args=training_args,                 # training arguments, defined above
    data_collator=collate_fn,           # the data collator that will be used for batching
    compute_metrics=compute_metrics,    # the metrics function that will be used for evaluation
    train_dataset=train_dataset,        # training dataset
    eval_dataset=val_dataset,           # evaluation dataset
    tokenizer=image_processor,          # the processor that will be used for preprocessing the images
)

In [ ]:
trainer.train()

***** Running training *****
  Num examples = 88,416
  Num Epochs = 40
  Instantaneous batch size per device = 32
  Total train batch size (w. parallel, distributed & accumulation) = 32
  Gradient Accumulation steps = 1
  Total optimization steps = 110,520
  Number of trainable parameters = 85,876,325


Step,Training Loss,Validation Loss,Accuracy,F1
9000,1.083000,1.230797,0.739063,0.749550
18000,0.205800,1.383399,0.749983,0.756967


***** Running Evaluation *****
  Num examples = 29486
  Batch size = 8
Saving model checkpoint to /content/MMLearning/data/Models/Model-13/checkpoint-9000
Configuration saved in /content/MMLearning/data/Models/Model-13/checkpoint-9000/config.json
Model weights saved in /content/MMLearning/data/Models/Model-13/checkpoint-9000/pytorch_model.bin
Image processor saved in /content/MMLearning/data/Models/Model-13/checkpoint-9000/preprocessor_config.json
***** Running Evaluation *****
  Num examples = 29486
  Batch size = 8
Saving model checkpoint to /content/MMLearning/data/Models/Model-13/checkpoint-18000
Configuration saved in /content/MMLearning/data/Models/Model-13/checkpoint-18000/config.json
Model weights saved in /content/MMLearning/data/Models/Model-13/checkpoint-18000/pytorch_model.bin
Image processor saved in /content/MMLearning/data/Models/Model-13/checkpoint-18000/preprocessor_config.json
***** Running Evaluation *****
  Num examples = 29486
  Batch size = 8


Buffered data was truncated after reaching the output size limit.

# Best Model Selection

In [ ]:
from torch.utils.data import DataLoader

batch_size = 32
valid_dataset_loader = DataLoader(test_dataset, collate_fn=collate_fn, batch_size=batch_size, shuffle=False)

In [ ]:
checkpoint = 9000

In [ ]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score, confusion_matrix, classification_report

while checkpoint <= 45000:
    model = ViTForImageClassification.from_pretrained(f"/content/Model/Models-Train-28/checkpoint-{checkpoint}").to(device)
    image_processor = ViTImageProcessor.from_pretrained(f"/content/Model/Models-Train-28/checkpoint-{checkpoint}")
    model = model.to(device)
    model.eval()
    predictions, labels = [], []

    for batch in valid_dataset_loader:
        pixel_values = batch["pixel_values"].to(device)
        label_ids = batch["labels"].to(device)

        outputs = model(pixel_values=pixel_values, labels=label_ids)
        logits = outputs.logits.detach().cpu()

        predictions.extend(logits.argmax(dim=-1).tolist())
        labels.extend(label_ids.tolist())

    accuracy = accuracy_score(labels, predictions)
    precision = precision_score(labels, predictions, average='weighted')
    recall = recall_score(labels, predictions, average='weighted')
    f1 = f1_score(labels, predictions, average='weighted')

    print("Accuracy: ", accuracy)
    print("precision: ", precision)
    print("f1_score: ", f1)
    print("recall", recall)
    print(classification_report(labels, predictions))
    checkpoint = checkpoint + 9000